### TOC

#### 1. [Load test-cad-2020.json](#1) 
#### 2. [Load test-neos-20202.csv](#2)
#### 3. [Saving data into a dictionary](#3)
#### 4. [Working with an iterator](#4)
#### 5. [Magic Methods](#5)
#### 6. [AttributeFilter Class](#6)
#### 7. [write.py test](#7)

***

#### 1. Load test-cad-2020.json<A id="1"><a/>

In [ ]:
import pandas as pd
import json 

#cad.json: close approaches
with open("test-cad-2020.json") as file:
    data=json.load(file)
    
df_cad=pd.DataFrame(data['data'], columns=data['fields'])
df_cad.head()

In [ ]:
data.keys()
data['fields']

In [ ]:
from collections import Counter

Counter(df_cad.des)

#### 2. Load test-neos-20202.csv<a id="2"></a>

In [ ]:
import pandas as pd

df_neos=pd.read_csv("test-neos-2020.csv", sep=",")
df_neos.head()

In [ ]:
df_neos.columns

In [ ]:
#df_neos.pdes
#df_neos[df_neos.name=="lemmon"]
df_neos[df_neos.name=="Lemmon"]

In [ ]:
df_neos.head()

In [ ]:
#set(df_neos.pha)

neos=sorted(set(df_neos.pdes))

In [ ]:
cad=sorted(set(df_cad.des))

In [ ]:
neos==cad

#### 3. Saving data into a dictionary<a id="3"></a>

In [ ]:
dictCAD={}

for i,des in enumerate(df_cad['des']):
    #create a dictionary of approaches by approach designation as the key
    if des not in dictCAD.keys():
        dictCAD[des]=[]
        dictCAD[des].append(df_cad.iloc[i,:])
    else:
        dictCAD[des].append(df_cad.iloc[i,:])

In [ ]:
dictNEO={}
dictNEO['CAD']=[]

for neo_des in df_neos['pdes']:
    print(dictCAD[neo_des])
    #dictNEO['CAD'].append(dictCAD[neo_des])
    

In [ ]:
dictNEO['CAD']

#### 4. Working with an iterator<a id="4"></a>
- https://www.w3schools.com/python/python_iterators.asp

In [ ]:
from itertools import islice

myIter=(i for i in range(50))
limit=10
sliced_myIter=islice(myIter, 0, limit)

In [ ]:
next(sliced_myIter)

In [ ]:
next(myIter)

#### 5. Magic Methods<a id="5"></a>
- Udacity, OOP, Part I, chpt. 13

In [ ]:
class MagicShoppingCart:
    """
    params:
    .items: a dictionary object
    """
    
    def __init__(self, items):
        self.items=items
        
    def __len__(self):
        return sum(self.items.values())
    
    def __str__(self):
        return f"MagicShoppingCart({self.items})"
    
    def __contains__(self,item):
        return item in self.items
    
    def __iadd__(self, other):
        for item, count in other.items.items():
            if item in self.items:
                self.items[item]+=count
            else:
                self.items[item]=count
        return self

In [ ]:
cart1=MagicShoppingCart({'apples':3, 'bananas':2})
cart2=MagicShoppingCart({'apples':2, 'oranges':5})

cart1+=cart2

print(cart1)

In [ ]:
myDict={'apples':2, 'oranges':5}
myDict.items.items()

In [ ]:
cart1.items.items()

#### 6. AttributeFilter Class<a id="6"></a>

In [ ]:
import operator 

class AttributeFilter:
    """A general superclass for filters on comparable attributes.

    An `AttributeFilter` represents the search criteria pattern comparing some
    attribute of a close approach (or its attached NEO) to a reference value. It
    essentially functions as a callable predicate for whether a `CloseApproach`
    object satisfies the encoded criterion.

    It is constructed with a comparator operator and a reference value, and
    calling the filter (with __call__) executes `get(approach) OP value` (in
    infix notation).

    Concrete subclasses can override the `get` classmethod to provide custom
    behavior to fetch a desired attribute from the given `CloseApproach`.
    """
    def __init__(self, op, value):
        """Construct a new `AttributeFilter` from an binary predicate and a reference value.

        The reference value will be supplied as the second (right-hand side)
        argument to the operator function. For example, an `AttributeFilter`
        with `op=operator.le` and `value=10` will, when called on an approach,
        evaluate `some_attribute <= 10`.

        :param op: A 2-argument predicate comparator (such as `operator.le`).
        :param value: The reference value to compare against.
        """
        self.op = op
        self.value = value

    def __call__(self, approach):
        """Invoke `self(approach)`."""
        return self.op(self.get(approach), self.value)

    @classmethod
    def get(cls, approach):
        """Get an attribute of interest from a close approach.

        Concrete subclasses must override this method to get an attribute of
        interest from the supplied `CloseApproach`.

        :param approach: A `CloseApproach` on which to evaluate this filter.
        :return: The value of an attribute of interest, comparable to `self.value` via `self.op`.
        """
        raise UnsupportedCriterionError

    def __repr__(self):
        return f"{self.__class__.__name__}(op=operator.{self.op.__name__}, value={self.value})"

    #>>>>>>>>>>>>>>>>>>>>>>
    def __le__(self, other):
        """less than equal to method
        """
        for item, count in other.items.items():
            if item in self.items:
                self.items[item]+=count
            else:
                self.items[item]=count
        return self


def create_filters(
        date=None, start_date=None, end_date=None,
        distance_min=None, distance_max=None,
        velocity_min=None, velocity_max=None,
        diameter_min=None, diameter_max=None,
        hazardous=None
):
    """Create a collection of filters from user-specified criteria.

    Each of these arguments is provided by the main module with a value from the
    user's options at the command line. Each one corresponds to a different type
    of filter. For example, the `--date` option corresponds to the `date`
    argument, and represents a filter that selects close approaches that occurred
    on exactly that given date. Similarly, the `--min-distance` option
    corresponds to the `distance_min` argument, and represents a filter that
    selects close approaches whose nominal approach distance is at least that
    far away from Earth. Each option is `None` if not specified at the command
    line (in particular, this means that the `--not-hazardous` flag results in
    `hazardous=False`, not to be confused with `hazardous=None`).

    The return value must be compatible with the `query` method of `NEODatabase`
    because the main module directly passes this result to that method. For now,
    this can be thought of as a collection of `AttributeFilter`s.

    :param date: A `date` on which a matching `CloseApproach` occurs.
    :param start_date: A `date` on or after which a matching `CloseApproach` occurs.
    :param end_date: A `date` on or before which a matching `CloseApproach` occurs.
    :param distance_min: A minimum nominal approach distance for a matching `CloseApproach`.
    :param distance_max: A maximum nominal approach distance for a matching `CloseApproach`.
    :param velocity_min: A minimum relative approach velocity for a matching `CloseApproach`.
    :param velocity_max: A maximum relative approach velocity for a matching `CloseApproach`.
    :param diameter_min: A minimum diameter of the NEO of a matching `CloseApproach`.
    :param diameter_max: A maximum diameter of the NEO of a matching `CloseApproach`.
    :param hazardous: Whether the NEO of a matching `CloseApproach` is potentially hazardous.
    :return: A collection of filters for use with `query`.
    """
    # TODO: Decide how you will represent your filters.

    if velocity_min != None:
        return
    
    #return ()

In [ ]:
import pandas as pd
import json 

#cad.json: close approaches
with open("test-cad-2020.json") as file:
    data=json.load(file)
    
df_cad=pd.DataFrame(data['data'], columns=data['fields'])
df_cad.head()

In [ ]:
import models

f=AttributeFilter(operator.eq, '433')
approach_433=models.CloseApproach(df_cad.iloc[0,:]["cd"], df_cad.iloc[0,:]["dist"], df_cad.iloc[0,:]["v_rel"])
f(approach_433)

***

In [ ]:
from filters import ObjectFilter

#allows to import the datasets to be filtered
from database import NEODatabase
from extract import load_neos, load_approaches

import operator

TEST_NEO_FILE = 'test-neos-2020.csv'
TEST_CAD_FILE = 'test-cad-2020.json'

#instantiate dataset objects
#load datasets
approaches=load_approaches(TEST_CAD_FILE)
neos=load_neos(TEST_NEO_FILE)

#instantiate filter objects
#approachfilter = ObjectFilter.get(approaches,operator.ge, '33')
approachfilter = ObjectFilter(approaches, operator.ge, '33')

approachfilter#.get(approaches);
#approachfilter.value

In [ ]:
#approachfilter.any_object

In [ ]:
objectfilter=ObjectFilter.get(approaches)
type(objectfilter)

In [ ]:
approachfilter.__dict__

In [ ]:
#approachfilter.get(load_approaches(TEST_CAD_FILE)) #check
#filter_hazardous=NEOFilter(operator.eq)

In [ ]:
import operator

operator.gt(5,10)

In [ ]:
myop=operator.gt
myop(20,17)

In [ ]:
from database import NEODatabase
from extract import load_neos, load_approaches
import operator

TEST_NEO_FILE = 'test-neos-2020.csv'
TEST_CAD_FILE = 'test-cad-2020.json'

#NEODatabase

neos=load_neos('test-neos-2020.csv')
approaches=load_approaches('test-cad-2020.json')

myDB=NEODatabase(neos,approaches)

In [ ]:
myDB._approaches

In [ ]:
from database import NEODatabase
from extract import load_neos, load_approaches
from filters import create_filters

TEST_NEO_FILE = 'test-neos-2020.csv'
TEST_CAD_FILE = 'test-cad-2020.json'

neos=load_neos('test-neos-2020.csv')
approaches=load_approaches('test-cad-2020.json')
myDB=NEODatabase(neos, approaches)

import datetime

date = datetime.date(2020, 3, 2)

filters = create_filters(date=date)
filters

In [ ]:
myList=[]
for approach in myDB._approaches:
    if approach.time.date()==filters['date']:
        myList.append(approach)

In [ ]:
received = set(myDB.query(filters))
received

In [ ]:
myDB._approaches[0].time.date()

In [ ]:
filters

In [ ]:
myDB.query(filters())

***

In [ ]:
setA={1,2,3,4}
setB={3,4,5,2}
setC=set()
setD={11,14,25,32}
total=set()

total=setA.intersection(setB)
total

In [ ]:
setC

In [ ]:
from collections import Counter

myDict={'A': 1, 'B':5, 'C':10, 'D': None, 'E': None, 'F':set(), 'G':set()}
#Counter(myDict)

for each in myDict.values():
    if each==set():
        print("Yes")

***

#### 7. write.py test<a id="7"></a>

In [27]:
import collections
import collections.abc
import contextlib
import csv
import datetime
import io
import json
import pathlib
import unittest
import unittest.mock


from extract import load_neos, load_approaches
from database import NEODatabase
from write import write_to_csv, write_to_json



TEST_NEO_FILE = 'test-neos-2020.csv'
TEST_CAD_FILE = 'test-cad-2020.json'


def build_results(n):
    neos = tuple(load_neos(TEST_NEO_FILE))
    approaches = tuple(load_approaches(TEST_CAD_FILE))

    # Only needed to link together these objects.
    NEODatabase(neos, approaches)

    return approaches[:n]

results=build_results(10)

In [35]:
type(results[0])

#dir(results[0])
list(results[0].serialize().values())+list(results[0].neo.serialize().values())

myDict=results[0].serialize()
myDict["neo"]=results[0].neo.serialize()
myDict

#type(results)
#dir(results[0].neo)

#'datetime_utc', 'distance_au', 'velocity_km_s',
#        'designation', 'name', 'diameter_km', 'potentially_hazardous'

{'datetime_utc': datetime.datetime(2020, 1, 1, 0, 54),
 'distance_au': 0.0211660525256395,
 'velocity_km_s': 5.62203195551878,
 'neo': {'designation': '2020 AY1',
  'name': None,
  'diameter_km': nan,
  'potentially_hazardous': False}}

In [36]:
row=results[0]
strRow=','.join([str(s) for s in [row.time.strftime("%Y-%m-%d %H:%M"), row.distance, row.velocity,\
                 row._designation, row.neo.name, row.neo.diameter, row.neo.hazardous]])
print(strRow)

2020-01-01 00:54,0.0211660525256395,5.62203195551878,2020 AY1,None,nan,False


In [37]:
row.time.strftime("%Y-%m-%d %H:%M")

'2020-01-01 00:54'

In [42]:
row.time.strftime("%Y-%m-%d %H:%M")

'2020-01-01 00:54'

In [4]:
results.serialize()

AttributeError: 'tuple' object has no attribute 'serialize'

In [2]:
import write

write_to_csv(results, "myoutput_10022022.csv")

TypeError: writerows() argument must be iterable

In [38]:
myList=[1,2,3,4]
','.join(myList)

TypeError: sequence item 0: expected str instance, int found

In [4]:
fin_ids = [1002774, 0, 1000702, 1000339, 
   1001620, 1000710, 1000202, 1003143, 147897, 
   31018, 1001502, 1002812, 1003026, 1003280, 
   1003289, 1002714, 133191, 5252218, 6007821, 1002632]
        
outfile = open("test_save_10022022.csv",'w')
out = csv.writer(outfile)
for each in fin_ids:
out.writerows(map(lambda x: [x], fin_ids))
outfile.close()

TypeError: writerows() argument must be iterable

In [4]:
neos = load_neos(TEST_NEO_FILE)

In [5]:
neos[0].serialize()

{'designation': '1685',
 'name': 'Toro',
 'diameter_km': 3.4,
 'potentially_hazardous': False}